### 원시 데이터인 df 넣어서 output => match, timelines 데이터가 있는 df 만들기
### 컬럼 = ['gameId','gameDuration','gameVersion','summonerName','summonerLevel','participantId','championName', 'champExperience','teamPosition','teamId','win','kills','deaths','assists', 'totalDamageDealtToChampions','totalDamageTaken']
### timeline_df [,'g5','g6' ~ 'g25'] => 25분까지 안 가는 게임 0으로 넣기
### pk 값 - (match_id, participantId)

In [1]:
import requests
import pandas as pd
from tqdm import tqdm
import my_utils as mu
import time
import random

In [2]:
riot_api_key = mu.riot_api_key
division_list = ['I', 'II', 'III', 'IV']

In [3]:
riot_api_key

'RGAPI-65c33668-88f0-4cf3-ad54-83f88fbdd49a'

In [ ]:
def get_rawdata(tier) :
    lst = []
    page = mu.rand()
    for i in range(len(division_list)) :
        url = f'https://kr.api.riotgames.com/lol/league/v4/entries/RANKED_SOLO_5x5/{tier}/{division_list[i]}?page=1&api_key={riot_api_key}'
        res = requests.get(url).json()
        lst += random.sample(res, 5)
    summoner_lst = list(map(lambda x : x['summonerName'], lst))
    df_lst = []
    for s in tqdm(summoner_lst) :
        puuid = mu.get_puuid(s)
        match_ids = mu.get_match_id(puuid, 3)
        for match_id in match_ids :
            matches, timelines = mu.get_matches_timelines(match_id)
            match_timelines_lst = [match_id, matches, timelines]
            df_lst.append(match_timelines_lst)
        time.sleep(3)
    df = pd.DataFrame(df_lst, columns=['match_id', 'matches', 'timelines'])
    return df

In [4]:
raw_data = mu.get_rawdata('GOLD')

100%|██████████| 20/20 [01:53<00:00,  5.69s/it]


In [ ]:
def get_match_timeline_df(df) :
    df_creator = []
    print('소환사 스텟 생성 중...')
    for i in range(len(df)) :
        print(i)
        if 'status' in raw_data.iloc[i]['matches'].keys() or 'status' in  raw_data.iloc[i]['timelines'].keys():
            print('status : 426')
        else :
            for j in range(len(raw_data.iloc[i]['matches']['info']['participants'])) :
                lst_match = []
                participantId = raw_data.iloc[i]['matches']['info']['participants'][j]['participantId']
                lst_match.append(raw_data.iloc[i]['matches']['info']['gameId'])
                lst_match.append(raw_data.iloc[i]['matches']['info']['gameDuration'])
                lst_match.append(raw_data.iloc[i]['matches']['info']['gameVersion'])
                lst_match.append(raw_data.iloc[i]['matches']['info']['participants'][j]['summonerName'])
                lst_match.append(raw_data.iloc[i]['matches']['info']['participants'][j]['summonerLevel'])
                lst_match.append(raw_data.iloc[i]['matches']['info']['participants'][j]['participantId'])
                lst_match.append(raw_data.iloc[i]['matches']['info']['participants'][j]['championName'])
                lst_match.append(raw_data.iloc[i]['matches']['info']['participants'][j]['champExperience'])
                lst_match.append(raw_data.iloc[i]['matches']['info']['participants'][j]['teamPosition'])
                lst_match.append(raw_data.iloc[i]['matches']['info']['participants'][j]['teamId'])
                lst_match.append(raw_data.iloc[i]['matches']['info']['participants'][j]['win'])
                lst_match.append(raw_data.iloc[i]['matches']['info']['participants'][j]['kills'])
                lst_match.append(raw_data.iloc[i]['matches']['info']['participants'][j]['deaths'])
                lst_match.append(raw_data.iloc[i]['matches']['info']['participants'][j]['assists'])
                lst_match.append(raw_data.iloc[i]['matches']['info']['participants'][j]['totalDamageDealtToChampions'])
                lst_match.append(raw_data.iloc[i]['matches']['info']['participants'][j]['totalDamageTaken'])
                print('participantId : ', participantId)
                if len(raw_data.iloc[i]['timelines']['info']['frames']) < 25 :
                    for k in range(4, 25) :
                        if k < len(raw_data.iloc[i]['timelines']['info']['frames']) :
                            lst_match.append(raw_data.iloc[i]['timelines']['info']['frames'][k]['participantFrames'][str(participantId)]['totalGold'])
                        else :
                            lst_match.append(0)
                else :
                    for k in range(4, 25) :
                        print('k : ', k)
                        lst_match.append(raw_data.iloc[i]['timelines']['info']['frames'][k]['participantFrames'][str(participantId)]['totalGold'])
                df_creator.append(lst_match)
    my_df = pd.DataFrame(df_creator, columns=['gameId', 'gameDuration', 'gameVersion', 'summonerName', 'summonerLevel', 'participantId', 'championName', 'champExperience', 'teamPosition', 'teamId', 'win', 'kills', 'deaths', 'assists', 'totalDamageDealtToChampions', 'totalDamageTaken', 'g5', 'g6', 'g7', 'g8', 'g9', 'g10', 'g11', 'g12', 'g13', 'g14', 'g15', 'g16', 'g17', 'g18', 'g19', 'g20', 'g21', 'g22', 'g23', 'g24', 'g25'])
    return my_df

In [47]:
lol_df = mu.get_match_timeline_df(raw_data)
#lol_df = get_match_timeline_df(raw_data)

AttributeError: module 'my_utils' has no attribute 'get_match_timeline_df'

In [ ]:
raw_data.iloc[9]['timelines'].keys()
len(raw_data.iloc[9]['timelines']['info']['frames'])
#raw_data.iloc[i]['timelines']['info']['frames'][9]['participantFrames'][str(participantId)]['totalGold']

In [ ]:
lol_df

In [ ]:
mu.db_open()

In [ ]:
query = """
CREATE TABLE LOL_PROJ(gameId VARCHAR(20), gameDuration VARCHAR(30), gameVersion VARCHAR(30), summonerName VARCHAR(80), summonerLevel NUMBER(20), participantId NUMBER(20), championName VARCHAR(30), champExperience NUMBER(20), teamPosition VARCHAR(30), teamId NUMBER(20), win VARCHAR(30), kills NUMBER(20), deaths NUMBER(20), assists NUMBER(20), totalDamageDealtToChampions NUMBER(20), totalDamageTaken NUMBER(20), g5 NUMBER(20), g6 NUMBER(20), g7 NUMBER(20), g8 NUMBER(20), g9 NUMBER(20), g10 NUMBER(20), g11 NUMBER(20), g12 NUMBER(20), g13 NUMBER(20), g14 NUMBER(20), g15 NUMBER(20), g16 NUMBER(20), g17 NUMBER(20), g18 NUMBER(20), g19 NUMBER(20), g20 NUMBER(20), g21 NUMBER(20), g22 NUMBER(20), g23 NUMBER(20), g24 NUMBER(20), g25 NUMBER(20), constraint m_pk primary key(gameId,participantId)
)
"""

In [ ]:
mu.oracle_execute(query)

In [ ]:
mu.oracle_close()

In [ ]:
def insert(df) :
    query = (
        f'insert into lol_proj (gameId, gameDuration, gameVersion, summonerName, summonerLevel, participantId, championName, champExperience, teamPosition, teamId, win, kills, deaths, assists, totalDamageDealtToChampions, totalDamageTaken, g5, g6, g7, g8, g9, g10, g11, g12, g13, g14, g15, g16, g17, g18, g19, g20, g21, g22, g23, g24, g25)'
        f' values ({repr(df.gameId)}, {repr(df.gameDuration)}, {repr(df.gameVersion)}, {repr(df.summonerName)}, {df.summonerLevel}, {df.participantId}, {repr(df.championName)}, {df.champExperience}, {repr(df.teamPosition)}, {df.teamId}, {repr(str(df.win))}, {df.kills}, {df.deaths}, {df.assists}, {df.totalDamageDealtToChampions}, {df.totalDamageTaken}, {df.g5}, {df.g6}, {df.g7}, {df.g8}, {df.g9}, {df.g10}, {df.g11}, {df.g12}, {df.g13}, {df.g14}, {df.g15}, {df.g16}, {df.g17}, {df.g18}, {df.g19}, {df.g20}, {df.g21}, {df.g22}, {df.g23}, {df.g24}, {df.g25})'
    )
    mu.oracle_execute(query)
    return

In [ ]:
mu.db_open()

In [ ]:
tqdm.pandas()
lol_df.progress_apply(lambda x : insert(x), axis = 1)

In [ ]:
mu.oracle_close()

In [ ]:
f'MERGE INTO lol_datas USING DUAL ON(gameId=\'{df.gameId}\' and participantId={df.participantId}) '
             f'WHEN NOT MATCHED THEN '
             f'insert